In [41]:
# Create model

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import numpy as np
import argparse
import pickle
import cv2
import os

# construct the argument parser and parse the arguments

epochs=20


# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["awake", "feeding", "sleeping"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images('data'))
print("imagePaths", imagePaths)
data = []
labels = []


[INFO] loading images...
imagePaths ['data\\awake\\awake (1).jpg', 'data\\awake\\awake (42).jpg', 'data\\awake\\awake (43).jpg', 'data\\awake\\awake (44).jpg', 'data\\awake\\awake (45).jpg', 'data\\awake\\awake (46).jpg', 'data\\awake\\awake (47).jpg', 'data\\awake\\awake (48).jpg', 'data\\awake\\awake (49).jpg', 'data\\awake\\awake (50).jpg', 'data\\awake\\awake (51).jpg', 'data\\awake\\awake (52).jpg', 'data\\awake\\awake (53).jpg', 'data\\awake\\awake (54).jpg', 'data\\awake\\awake (55).jpg', 'data\\awake\\awake (56).jpg', 'data\\awake\\awake (57).jpg', 'data\\awake\\awake (58).jpg', 'data\\awake\\awake (59).jpg', 'data\\awake\\awake (60).jpg', 'data\\awake\\awake (61).jpg', 'data\\awake\\awake (62).jpg', 'data\\awake\\awake (63).jpg', 'data\\awake\\awake (64).jpg', 'data\\awake\\awake (65).jpg', 'data\\awake\\awake (66).jpg', 'data\\awake\\awake (67).jpg', 'data\\awake\\awake (68).jpg', 'data\\awake\\awake (69).jpg', 'data\\awake\\awake (70).jpg', 'data\\awake\\awake (71).jpg', 'da

In [42]:

# loop over the image paths
for imagePath in imagePaths:
    
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # if the label of the current image is not part of of the labels
    # are interested in, then ignore the image
    if label not in LABELS:
        continue

            
        
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (300, 300))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.25, stratify=labels, random_state=42)

# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(300, 300, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
    x=trainAug.flow(trainX, trainY, batch_size=32),
    steps_per_epoch=len(trainX) // 32,
    validation_data=valAug.flow(testX, testY),
    validation_steps=len(testX) // 32,
    epochs=epochs)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX.astype("float32"), batch_size=4)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = epochs

[INFO] compiling model...
[INFO] training head...
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3/3 [==============================] - ETA: 0s - loss: 1.8784 - accuracy: 0.2500WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000017685C7D5E8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Li

c:\programdata\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [44]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save('model.h5')

# serialize the label binarizer to disk
print("[INFO] loading...")
f = open('lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network...
[INFO] loading...


In [46]:
#predict sample videos
# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
import plotly.graph_objects as go
import numpy as np


time_skips = float(1000) #skip every 1 seconds. You need to modify this
count = 0

predic_sleeping = []
predic_feeding = []
predic_awake = []
time_sec=[]

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
#ap.add_argument("-m", "--model", required=True,help="path to trained serialized model")
#ap.add_argument("-l", "--label-bin", required=True,help="path to  label binarizer")
#ap.add_argument("-i", "--input", required=True,help="path to our input video")
#ap.add_argument("-o", "--output", required=True,help="path to our output video")
#ap.add_argument("-s", "--size", type=int, default=128,help="size of queue for averaging")
#args = vars(ap.parse_args())
#args["size"]=queue
queuesize=128

#args["size"]=queue
queue=128

# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model('model.h5')
lb = pickle.loads(open('lb.pickle', "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=queuesize)

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture('sample/sample1.mp4')
writer = None
(W, H) = (None, None)
print(lb.classes_)
# loop over frames from the video file stream
while True:
    
    vs.set(cv2.CAP_PROP_POS_MSEC,(count*time_skips))    # move the time
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    
    #360, 640, 3
    # clone the output frame, then convert it from BGR to RGB
    # ordering, resize the frame to a fixed 224x224, and then
    # perform mean subtraction
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    ##
    #frame=cv2.GaussianBlur(frame, (5, 5), 0)
    frame = cv2.resize(frame, (300, 300)).astype("float32")
    frame -= mean
    # make predictions on the frame and then update the predictions
    # queue
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    print(preds)
    predic_awake.append(preds[0])
    print(predic_awake)
    predic_feeding.append(preds[1])
    predic_sleeping.append(preds[2])
    time_sec.append(count)
    
    count += 1
    
    #print(preds)
    Q.append(preds)

    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]

    # draw the activity on the output frame
    
    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
        1.25, (0, 255, 0), 5)

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("Output/", fourcc, 30,
            (H, W), True)

    # write the output frame to disk
    writer.write(output)

    # show the output image
    #cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# release the file pointers
print("[INFO] cleaning up...")
#writer.release()
#vs.release()


#################################
##### This plots the data #######
#################################


fig = go.Figure()
fig.add_trace(go.Scatter(x=time_sec, y=predic_awake,
                    mode='lines',
                    name='active'))
fig.add_trace(go.Scatter(x=time_sec, y=predic_feeding,
                    mode='lines',
                    name='eating'))
fig.add_trace(go.Scatter(x=time_sec, y=predic_sleeping,
                    mode='lines',
                    name='sleeping'))

fig.show()


print(Q)

[INFO] loading model and label binarizer...
['awake' 'feeding' 'sleeping']
[INFO] cleaning up...


deque([], maxlen=128)


In [38]:
results

NameError: name 'results' is not defined